In [65]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [66]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [67]:
pip install --upgrade nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from time import time

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from joblib import dump
from sklearn.inspection import permutation_importance

### Подготовка данных

Загрузим все необходимые данные

In [2]:
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [3]:
movies_df = pd.read_csv("movies.csv")

In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212973 entries, 0 to 212972
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Sentence  212344 non-null  object
 1   Movie     212973 non-null  object
 2   Level     212973 non-null  object
dtypes: object(3)
memory usage: 4.9+ MB


In [5]:
print(movies_df.head())

                                            Sentence  \
0                                        little girl   
1                 don't be afraid, okay? little girl   
2                         panting growling oh my god   
3  theme music playing police radio chatter man w...   
4  man i never met a woman who knew how to turn o...   

                                           Movie Level  
0  The Walking Dead-S01E01-Days Gone Bye.English    A2  
1  The Walking Dead-S01E01-Days Gone Bye.English    A2  
2  The Walking Dead-S01E01-Days Gone Bye.English    A2  
3  The Walking Dead-S01E01-Days Gone Bye.English    A2  
4  The Walking Dead-S01E01-Days Gone Bye.English    A2  


Данные загружены. У нас есть столбцы с текстами из фильмов, название и сам уровень. Можно приступать к предобработке.

### Предобработка

In [6]:
# удалим дубликаты 
movies_df = movies_df.drop_duplicates()
movies_df.shape

(197772, 3)

In [7]:
movies_df.isna().sum()

Sentence    107
Movie         0
Level         0
dtype: int64

In [8]:
# Удаление пропущенных значений
movies_df = movies_df.dropna(subset=["Sentence"])

Для более-менее сбалансированной выборки возьмём определенное количество текстов каждого уровня.

In [9]:
# Создание новой таблицы с заданным количеством строк для каждого уровня
num_rows_per_level = 37000
levels = ['A2', 'B1', 'B2', 'C1']
new_table = pd.DataFrame(columns=['Sentence', 'Level'])

In [10]:
for level in levels:
    level_rows = movies_df[movies_df['Level'] == level].head(num_rows_per_level)
    new_table = new_table.append(level_rows, ignore_index=True)

In [11]:
movies_df = new_table

In [12]:
print(movies_df['Level'].value_counts())

B1    37000
B2    37000
C1    18595
A2     2630
Name: Level, dtype: int64


In [13]:
def get_unique_lemmas_by_level(df):
    lemmatizer = WordNetLemmatizer()
    levels = ['A1', 'A2', 'B1', 'B2', 'C1']
    unique_lemmas_by_level = {level: [] for level in levels}
    
    for _, row in df.iterrows():
        text = row['Sentence'].lower()
        level = row['Level']
        tokens = nltk.word_tokenize(text)
        lemmas = [lemmatizer.lemmatize(token) for token in tokens]
        unique_lemmas = len(set(lemmas))
        unique_lemmas_by_level[level].append(unique_lemmas)
    
    return unique_lemmas_by_level

In [14]:
# Вызов функции get_unique_lemmas_by_level и получение словаря с количеством уникальных лемм для каждого уровня
unique_lemmas = get_unique_lemmas_by_level(movies_df)

In [15]:
# определим словарь меток с кодированием значений
label_dict = {'A2': 1,
              'B1': 2,
              'B2': 3,
              'C1': 4}
# заменим метки числовыми значениями
movies_df = movies_df.replace(label_dict)

In [16]:
# Функция для подсчета слов каждого уровня
def count_word_levels(sentence):
    words = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)

    level_counts = {"A2": 0, "B1": 0, "B2": 0, "C1": 0}

    for word, tag in pos_tags:
        if tag.startswith("NN"):
            level_counts["A2"] += 1
        elif tag.startswith("VB"):
            level_counts["B1"] += 1
        elif tag.startswith("JJ"):
            level_counts["B2"] += 1
        elif tag.startswith("RB"):
            level_counts["C1"] += 1

    return level_counts

In [17]:
for level in unique_lemmas:
    column_name = f'{level}_unique_lemmas'
    unique_lemmas_level = unique_lemmas[level]
    movies_df[column_name] = pd.Series(unique_lemmas_level).fillna(0)

C:\Users\HP\AppData\Local\Temp\ipykernel_3572\1894916258.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  movies_df[column_name] = pd.Series(unique_lemmas_level).fillna(0)


In [18]:
def lemma_count(lemmas, oxf, cat):
    func_dict = {'A1': 0,
                 'A2': 1,
                 'B1': 2,
                 'B2': 3,
                 'C1': 4}
    level = func_dict[cat]
    oxf_word_list = oxf[level].split()
    words = [lemma for lemma in lemmas if lemma in oxf_word_list]

    return len(set(words))

In [19]:
# Функция для токенизации предложения
def tokenize_sentence(sentence):
    tokens = word_tokenize(sentence)
    return tokens

In [20]:
# Применение функции count_word_levels для подсчета слов каждого уровня в предложениях
movies_df[["A2", "B1", "B2", "C1"]] = pd.DataFrame(movies_df["Sentence"].apply(count_word_levels).tolist())

In [21]:
# Функция для подсчета количества существительных в предложении
def count_nouns(sentence):
    tokens = word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    noun_count = len([word for word, pos in tagged if pos.startswith('NN')])
    return noun_count

In [22]:
movies_df['Noun_Count'] = movies_df['Sentence'].apply(count_nouns)

In [23]:
all_stopwords = stopwords.words('english')
porter = PorterStemmer()

def tokenizer_porter(text):
    if text == text:
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = word_tokenize(text)
        words = [word for word in words if word.casefold() not in all_stopwords]

        return " ".join([porter.stem(word) for word in words])
    else:
        return None

In [24]:
movies_df = movies_df.drop('A1_unique_lemmas', axis=1)

In [25]:
# выполняем нормализацию данных
movies_df['porter_text'] = movies_df['Sentence'].apply(tokenizer_porter)

movies_df.head()

,Sentence,Level,Movie,A2_unique_lemmas,B1_unique_lemmas,B2_unique_lemmas,C1_unique_lemmas,A2,B1,B2,C1,Noun_Count,porter_text
0,little girl,1,The Walking Dead-S01E01-Days Gone Bye.English,2.0,15.0,24.0,6.0,1,0,1,0,1,littl girl
1,"don't be afraid, okay? little girl",1,The Walking Dead-S01E01-Days Gone Bye.English,9.0,4.0,11.0,7.0,1,3,2,1,1,dont afraid okay littl girl
2,panting growling oh my god,1,The Walking Dead-S01E01-Days Gone Bye.English,5.0,10.0,14.0,4.0,1,2,1,0,1,pant growl oh god
3,theme music playing police radio chatter man w...,1,The Walking Dead-S01E01-Days Gone Bye.English,24.0,9.0,14.0,8.0,17,4,1,0,17,theme music play polic radio chatter man what ...
4,man i never met a woman who knew how to turn o...,1,The Walking Dead-S01E01-Days Gone Bye.English,13.0,8.0,6.0,12.0,3,4,0,1,3,man never met woman knew turn light


In [26]:
movies_df = movies_df.fillna(0)

In [27]:
movies_df

,Sentence,Level,Movie,A2_unique_lemmas,B1_unique_lemmas,B2_unique_lemmas,C1_unique_lemmas,A2,B1,B2,C1,Noun_Count,porter_text
0,little girl,1,The Walking Dead-S01E01-Days Gone Bye.English,2.0,15.0,24.0,6.0,1,0,1,0,1,littl girl
1,"don't be afraid, okay? little girl",1,The Walking Dead-S01E01-Days Gone Bye.English,9.0,4.0,11.0,7.0,1,3,2,1,1,dont afraid okay littl girl
2,panting growling oh my god,1,The Walking Dead-S01E01-Days Gone Bye.English,5.0,10.0,14.0,4.0,1,2,1,0,1,pant growl oh god
3,theme music playing police radio chatter man w...,1,The Walking Dead-S01E01-Days Gone Bye.English,24.0,9.0,14.0,8.0,17,4,1,0,17,theme music play polic radio chatter man what ...
4,man i never met a woman who knew how to turn o...,1,The Walking Dead-S01E01-Days Gone Bye.English,13.0,8.0,6.0,12.0,3,4,0,1,3,man never met woman knew turn light
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95220,every charge i'm about to bring against your m...,4,The_Legend_of_Tarzan(2016),0.0,0.0,0.0,0.0,4,5,2,1,4,everi charg im bring majesti person govern con...
95221,"we have witnesses, documents, letters, and off...",4,The_Legend_of_Tarzan(2016),0.0,0.0,0.0,0.0,9,3,3,0,9,wit document letter offici record bring attent...
95222,"july th,",4,The_Legend_of_Tarzan(2016),0.0,0.0,0.0,0.0,2,0,0,0,2,juli th
95223,jane they are singing the legend of tarzan,4,The_Legend_of_Tarzan(2016),0.0,0.0,0.0,0.0,3,2,0,0,3,jane sing legend tarzan


### Обучение моделей

In [28]:
features = movies_df[['porter_text', 'A2', 'B1', 'B2', 'C1', 'A2_unique_lemmas', 'B1_unique_lemmas', 'B2_unique_lemmas', 'C1_unique_lemmas', 'Noun_Count']]
target = movies_df['Level']

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345, shuffle=True)

print(f'Обучающая выборка:', features_train.shape[0])
print(f'Валидационная выборка:', features_valid.shape[0])

Обучающая выборка: 71418
Валидационная выборка: 23807


In [29]:
numeric = ['A2', 'B1', 'B2', 'C1']

features_train_ohe = features_train.copy()
features_valid_ohe = features_valid.copy()

scaler = MinMaxScaler()
features_train_ohe[numeric] = scaler.fit_transform(features_train_ohe[numeric])
features_valid_ohe[numeric] = scaler.transform(features_valid_ohe[numeric])

In [30]:
# используем викторизацию
tfidf = TfidfVectorizer(lowercase=False, max_features=10000, ngram_range=(1, 3))
column_transformer = ColumnTransformer([('vect1', tfidf, 'porter_text')], remainder='passthrough')

features_train_ohe = column_transformer.fit_transform(features_train_ohe)
features_valid_ohe = column_transformer.transform(features_valid_ohe)

In [31]:
best_score = 0
best_cls = None
best_params = None

In [32]:
def benchmark(clf, params, name):
    print("=" * 80)
    print(name)
    print(params)
    
    start_time = time()
    
    gs_clf = GridSearchCV(clf, params, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
    gs_clf.fit(features_train_ohe, target_train)
    
    train_time = time() - start_time
    print(f"Train time: {train_time:.3}s")

    start_time = time()
    
    best_clf = gs_clf.best_estimator_
    pred = best_clf.predict(features_valid_ohe)
    test_time = time() - start_time
    print(f"Test time:  {test_time:.3}s")

    score = metrics.f1_score(target_valid, pred, average='weighted')
    
    global best_cls, best_score, best_params
    
    if score > best_score:
        best_cls = best_clf
        best_score = score
        best_params = gs_clf.best_params_
    
    print(f"Valid accuracy:   {score:.3}")
    
    if hasattr(clf, "coef_"):
        print(f"Dimensionality: {best_clf.coef_.shape[1]}")
        print(f"Density: {best_clf.coef_.density}")
    
    print()
    
    return name, score, train_time, test_time


In [33]:
results = []
STATE = 12345

classifiers = [
    (LinearSVC(class_weight='balanced', loss='squared_hinge', dual=False, random_state=STATE), 
     {'C': [0.1, 0.5, 1.0]}, "Linear SVC"),
    (SGDClassifier(alpha=1e-4, early_stopping=True, class_weight='balanced', random_state=STATE, n_jobs=-1), 
     {'n_iter_no_change': [3, 5, 7], 'loss': ['hinge', 'log_loss', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']}, "SGD"),
    (LGBMClassifier(random_state=STATE, verbose=1), {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01, 0.001]}, "LightGBM")
]

for clf, params, name in classifiers:
    results.append(benchmark(clf, params, name))

Linear SVC
{'C': [0.1, 0.5, 1.0]}
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Train time: 40.9s
Test time:  0.006s
Valid accuracy:   0.816

SGD
{'n_iter_no_change': [3, 5, 7], 'loss': ['hinge', 'log_loss', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']}
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Train time: 49.2s
Test time:  0.004s
Valid accuracy:   0.802

LightGBM
{'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01, 0.001]}
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.682583 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 84085
[LightGBM] [Info] Number of data points in the train set: 71418, number of used features: 2663
[LightGBM] [Info] Start training from score -3.608960
[LightGBM] [Info] Start train

In [34]:
# Вывод результатов
print("=" * 80)
print("Результаты:")
for result in results:
    name, score, train_time, test_time = result
    print(f"Модель: {name}")
    print(f"Valid F1: {score:.3f}")
    print(f"Train time: {train_time:.3f}s")
    print(f"Test time: {test_time:.3f}s")
    print()

# Вывод наилучшей модели
print("=" * 80)
print("Наилучшая модель:")
print(f"Модель: {best_cls.__class__.__name__}")
print(f"Valid F1: {best_score:.3f}")
print("Лучшие параметры:", best_params)

Результаты:
Модель: Linear SVC
Valid F1: 0.816
Train time: 40.883s
Test time: 0.006s

Модель: SGD
Valid F1: 0.802
Train time: 49.221s
Test time: 0.004s

Модель: LightGBM
Valid F1: 0.789
Train time: 616.083s
Test time: 0.575s

Наилучшая модель:
Модель: LinearSVC
Valid F1: 0.816
Лучшие параметры: {'C': 0.1}


In [35]:
dump(best_cls, 'linear_svc_model.joblib')

['linear_svc_model.joblib']

In [47]:
# Преобразование разреженной матрицы в плотную
features_train_dense = features_train_ohe.toarray()

# Вычисление перестановочной важности
result = permutation_importance(best_cls, features_train_dense, target_train, n_repeats=10, random_state=42)

# Получение отсортированных признаков по важности
sorted_indices = result.importances_mean.argsort()[::-1]
sorted_feature_names = [feature_names[i] for i in sorted_indices]
sorted_importances = result.importances_mean[sorted_indices]

# Вывод важности признаков
for feature_name, importance in zip(sorted_feature_names, sorted_importances):
    print(f"{feature_name}: {importance}")

Исходя из результатов, лучшей моделью оказалась модель LinearSVC с валидационной оценкой F1 0.816. Она достигла этой оценки при использовании параметра C равного 0.1.

Время обучения модели LinearSVC составило 37.927 секунд, что является относительно небольшим временем для обучения. Время тестирования модели было всего 0.011 секунд.

С другой стороны, модель SGD также показала хорошие результаты с валидационной оценкой F1 0.802. Время обучения для модели SGD составило 47.084 секунды, а время тестирования было 0.004 секунды.

Модель LightGBM демонстрирует немного более низкую валидационную оценку F1 в 0.789, однако она обладает наиболее длительным временем обучения, равным 623.204 секунды, и временем тестирования, составляющим 0.593 секунды.

Исходя из этих результатов, можно заключить, что модель LinearSVC с параметром C равным 0.1 является наилучшей моделью с высокой оценкой F1 и относительно небольшим временем обучения и тестирования.